In [1]:
using StatisticalRethinking
using CmdStan, StanMCMCChain

ProjDir = rel_path("..", "scripts", "12")

d = CSV.read(rel_path( "..", "data",  "Kline.csv"), delim=';');
size(d) # Should be 10x5

(10, 5)

New col logpop, set log() for population data

In [2]:
d[:logpop] = map((x) -> log(x), d[:population]);
d[:society] = 1:10;

first(d, 5)

m12_6_1 = "
data{
    int total_tools[10];
    real logpop[10];
    int society[10];
}
parameters{
    real a;
    real bp;
    vector[10] a_society;
    real<lower=0> sigma_society;
}
model{
    vector[10] mu;
    sigma_society ~ cauchy( 0 , 1 );
    a_society ~ normal( 0 , sigma_society );
    bp ~ normal( 0 , 1 );
    a ~ normal( 0 , 10 );
    for ( i in 1:10 ) {
        mu[i] = a + a_society[society[i]] + bp * logpop[i];
        mu[i] = exp(mu[i]);
    }
    total_tools ~ poisson( mu );
}
";

Define the Stanmodel and set the output format to :mcmcchain.

In [3]:
stanmodel = Stanmodel(name="m12.6.1",  model=m12_6_1, num_warmup=2000,
num_samples=2000, output_format=:mcmcchain);

Input data for cmdstan

In [4]:
m12_6_1_data = Dict("total_tools" => d[:total_tools], "logpop" => d[:logpop], "society" => d[:society]);

Sample using cmdstan

In [5]:
rc, chn, cnames = stan(stanmodel, m12_6_1_data, ProjDir, diagnostics=false, CmdStanDir=CMDSTAN_HOME);


Inference for Stan model: m12_6_1_model
4 chains: each with iter=(2000,2000,2000,2000); warmup=(0,0,0,0); thin=(1,1,1,1); 8000 iterations saved.

Warmup took (0.93, 1.00, 1.0, 0.86) seconds, 3.8 seconds total
Sampling took (1.2, 1.0, 1.1, 1.2) seconds, 4.5 seconds total

                    Mean     MCSE   StdDev        5%       50%       95%    N_Eff  N_Eff/s    R_hat
lp__             9.3e+02  7.4e-02  3.1e+00   9.2e+02   9.3e+02   9.3e+02  1.7e+03  3.8e+02  1.0e+00
accept_stat__    9.5e-01  7.6e-04  7.2e-02   7.9e-01   9.8e-01   1.0e+00  9.1e+03  2.0e+03  1.0e+00
stepsize__       5.0e-02  4.3e-03  6.1e-03   4.6e-02   4.8e-02   6.1e-02  2.0e+00  4.5e-01  3.6e+13
treedepth__      5.9e+00  5.6e-02  4.6e-01   5.0e+00   6.0e+00   6.0e+00  6.7e+01  1.5e+01  1.0e+00
n_leapfrog__     7.5e+01  2.0e+00  3.7e+01   3.1e+01   6.3e+01   1.3e+02  3.3e+02  7.4e+01  1.0e+00
divergent__      0.0e+00      nan  0.0e+00   0.0e+00   0.0e+00   0.0e+00      nan      nan      nan
energy__        -9.2e+02  8

Describe the draws

In [6]:
describe(chn)

Iterations = 1:2000
Thinning interval = 1
Chains = 1,2,3,4
Samples per chain = 2000

Empirical Posterior Estimates:
                   Mean            SD         Naive SE        MCSE         ESS   
         lp__  927.898881625  3.0708168483 0.03433277610 0.07336687758 1751.89372
accept_stat__    0.948275137  0.0719938305 0.00080491550 0.00078319236 2000.00000
   stepsize__    0.050429125  0.0060792345 0.00006796791 0.00068392457   79.00988
  treedepth__    5.908000000  0.4577796362 0.00511813193 0.01405391967 1061.00640
 n_leapfrog__   74.920000000 36.7845992005 0.41126432169 0.86897233063 1791.92637
  divergent__    0.000000000  0.0000000000 0.00000000000 0.00000000000        NaN
     energy__ -921.389650875  3.9956760543 0.04467301637 0.08701980609 2000.00000
            a    1.083071693  0.7245945181 0.00810121299 0.01709971851 1795.61062
           bp    0.262501374  0.0782145084 0.00087446479 0.00180773255 1872.00189
  a_society.1   -0.196678372  0.2416099641 0.00270128152 0.00431

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*